In [71]:
#Import necessary modules
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import col, when, lag, round, sqrt, pow, sum as sum_func, max as max_func,row_number, lit, min as min_func, abs
from pyspark.sql.types import StructType, StructField, LongType, StringType, BooleanType, DoubleType, IntegerType
from pyspark.sql.functions import monotonically_increasing_id

In [72]:
#Create spark session
spark = SparkSession.builder.appName("Formation insights").getOrCreate()

In [73]:
#Select only desired columns and apply schema
schema = StructType([
    StructField("gameId", LongType(), True),
    StructField("playId", LongType(), True),
    StructField("nflId", DoubleType(), True),
    StructField("frameId", LongType(), True),
    StructField("x", DoubleType(), True),
    StructField("y", DoubleType(), True),
    StructField("frameType", StringType(), True)
])

#Merge all the tracking data into one dataframe.
trackingDf = spark.read.schema(schema).parquet("data/tracking/").cache()

#Filter and select tracking data just before the snap - to determine if there is any pre-snap motion
snapEventsDf = trackingDf.filter(col("frameType") == "SNAP").groupby("gameId", "playId").agg(min_func("frameId").alias("snapEvent"))

#Join with full data to get last second before snap, using the last 10 frames (update rate of 0.1 seconds)
presnapDf = trackingDf.join(snapEventsDf, ["gameId", "playId"]).filter(col("frameId") < col("snapEvent")).filter(col("frameId") >= (col("snapEvent") - 10))

25/06/06 15:02:47 WARN CacheManager: Asked to cache already cached data.


In [74]:
#Read in columns of interest from players data source 
schema = StructType([
    StructField("nflId", LongType(), True), 
    StructField("position", StringType(), True)
])

#Read in players datasource and apply schema
playersDf = spark.read.schema(schema).parquet("data/players.parquet").cache()

#Add positions column, through a join
presnapDf = presnapDf.join(playersDf.select("nflId", "position"), on="nflId", how="left")

playersDf.unpersist()

#Position classifications
offensivePositions = ["QB", "RB", "FB", "HB", "WR", "TE", "LT", "LG", "C", "RG", "RT"]
defensivePositions = ["CB", "S", "FS", "SS", "MLB", "OLB", "ILB", "LB", "DT", "DE", "NT", "DB"]

#Create a classifier column for if the player is on offence or defence
presnapDf = presnapDf.withColumn("isOffence",when(col("position").isin(*offensivePositions), True).when(col("position").isin(*defensivePositions), False).otherwise(lit(None)).cast(BooleanType()))

safetyDf = presnapDf.filter(col("position").isin(["SS", "FS"]))

schema = StructType([
    StructField("gameId", LongType(), True),
    StructField("playId", LongType(), True),
    StructField("yardlineNumber", LongType(), True)
])

playsDf = spark.read.schema(schema).parquet("data/plays.parquet").cache()

safetyDf = safetyDf.join(playsDf.select("gameId", "playId", "yardlineNumber"), on = ["gameId", "playId"], how = "left")

safetyDf = safetyDf.withColumn("distance", abs(col("yardlineNumber") - col("x")))

frameGroupByDf = safetyDf.groupBy("nflId", "gameId", "playId").agg(max_func("frameId"))

safetyDf = safetyDf.join(frameGroupByDf, on = ["nflId","gameId", "playId"], how = "inner")

safetyDf.show(5)

#Drop redundant columns
presnapDf = presnapDf.drop("frameId", "frameType", "snapEvent", "position")

presnapDf = presnapDf.withColumn("sequenceId", monotonically_increasing_id())
sparkWindow = Window.partitionBy("gameId", "playId", "nflId").orderBy("sequenceId")

#Add orderId using row number (for later sorting)
presnapDf = presnapDf.withColumn("orderId", row_number().over(sparkWindow))

presnapDf = presnapDf.withColumn("prevX", round(lag("x").over(sparkWindow), 2))
presnapDf = presnapDf.withColumn("prevY", round(lag("y").over(sparkWindow), 2))

#Calculate cumulative distance
presnapDf = presnapDf.withColumn("cumulativeDistance", round(when(col("prevX").isNull() | col("prevY").isNull(), 0.0).otherwise(sqrt(pow(col("x") - col("prevX"), 2) + pow(col("y") - col("prevY"), 2))),5))

#Sum all the movement values - this gives total distance moved in a second
playerMotion = presnapDf.groupBy("gameId", "playId", "nflId", "isOffence").agg(sum_func("cumulativeDistance").alias("distanceMoved"))

presnapDf.unpersist()

#Using a threshold of 2 yards in the 1 second time frame, determine if the player was in motion
playerMotion = playerMotion.withColumn("motion", col("distanceMoved") > 2)

#Determine if any player on each side was in motion
playMotion = playerMotion.groupBy("gameId", "playId", "isOffence").agg(max_func("motion").alias("isMotion"))

#Create final result
playMotion = playMotion.select("gameId", "playId", "isOffence", "isMotion")

25/06/06 15:02:47 WARN CacheManager: Asked to cache already cached data.


25/06/06 15:02:48 WARN MemoryStore: Not enough space to cache rdd_4_1 in memory! (computed 65.8 MiB so far)
25/06/06 15:02:48 WARN MemoryStore: Not enough space to cache rdd_4_0 in memory! (computed 65.8 MiB so far)
25/06/06 15:02:54 WARN MemoryStore: Not enough space to cache rdd_4_2 in memory! (computed 101.1 MiB so far)
25/06/06 15:02:58 WARN MemoryStore: Not enough space to cache rdd_4_4 in memory! (computed 101.0 MiB so far)
25/06/06 15:02:58 WARN MemoryStore: Not enough space to cache rdd_4_5 in memory! (computed 42.7 MiB so far)
25/06/06 15:03:02 WARN MemoryStore: Not enough space to cache rdd_4_7 in memory! (computed 101.1 MiB so far)
25/06/06 15:03:06 WARN MemoryStore: Not enough space to cache rdd_4_0 in memory! (computed 42.7 MiB so far)
25/06/06 15:03:06 WARN MemoryStore: Not enough space to cache rdd_4_8 in memory! (computed 101.1 MiB so far)
25/06/06 15:03:10 WARN MemoryStore: Not enough space to cache rdd_4_1 in memory! (computed 65.8 MiB so far)
25/06/06 15:03:10 WARN M

+-------+----------+------+-------+-----+-----+-----------+---------+--------+---------+--------------+------------------+------------+
|  nflId|    gameId|playId|frameId|    x|    y|  frameType|snapEvent|position|isOffence|yardlineNumber|          distance|max(frameId)|
+-------+----------+------+-------+-----+-----+-----------+---------+--------+---------+--------------+------------------+------------+
|52542.0|2022092511|  1986|     16|15.24|36.28|BEFORE_SNAP|       26|      FS|    false|            13|              2.24|          25|
|52542.0|2022092511|  1986|     17|15.31|36.31|BEFORE_SNAP|       26|      FS|    false|            13|2.3100000000000005|          25|
|52542.0|2022092511|  1986|     18|15.37|36.34|BEFORE_SNAP|       26|      FS|    false|            13| 2.369999999999999|          25|
|52542.0|2022092511|  1986|     19|15.42|36.36|BEFORE_SNAP|       26|      FS|    false|            13|              2.42|          25|
|52542.0|2022092511|  1986|     20|15.47|36.37|B